In [29]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.utils import to_categorical
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [30]:
# Directorio donde se encuentran tus archivos CSV
directory = r'C:\Users\Asus\Documents\aUniversidad\SEMESTRE 23-2\Bio_I\Proyecto'

# Mapeo de letras a nombres de archivo y contadores para cada tipo de movimiento
file_names = {'R': 'RIGHT_.csv', 'L': 'LEFT_.csv', 'U': 'UP_.csv', 'D': 'DOWN_.csv', 'P': 'PRESS_.csv'}


# Lista para almacenar datos normalizados por sesión
normalized_data_by_session = []

for letter, file_name in file_names.items():
    file_path = os.path.join(directory, file_name)
    if os.path.exists(file_path):
        # Leer el archivo CSV por sesión
        data_df = pd.read_csv(file_path)
        
        # Normalizar los datos dentro de cada sesión
        scaler = StandardScaler()
        data_df[['horizontal', 'vertical']] = scaler.fit_transform(data_df[['horizontal', 'vertical']])
        
        # Agregar datos normalizados por sesión a la lista
        normalized_data_by_session.append(data_df)

# Combinar todos los datos normalizados por sesión en un solo DataFrame
all_data_df_by_session = pd.concat(normalized_data_by_session, ignore_index=True)


In [31]:
# Separar los datos por sesión
sessions = all_data_df_by_session['label'].unique()
for session in sessions:
    session_data = all_data_df_by_session[all_data_df_by_session['label'] == session]
    
    # Preparar los datos de entrada y salida
    X_session = session_data[['horizontal', 'vertical']].values
    y_session = to_categorical(session_data['label'].values)
    
    # Dividir los datos en entrenamiento y prueba
    X_train_session, X_test_session, y_train_session, y_test_session = train_test_split(
        X_session, y_session, test_size=0.2, random_state=42
    )
    
    # Redimensionar los datos para la LSTM
    X_train_session = X_train_session.reshape(X_train_session.shape[0], 1, X_train_session.shape[1])
    X_test_session = X_test_session.reshape(X_test_session.shape[0], 1, X_test_session.shape[1])


In [32]:
# Lista para almacenar los modelos entrenados por sesión
trained_models_by_session = []

# Iterar sobre cada sesión
for session in sessions:
    session_data = all_data_df_by_session[all_data_df_by_session['label'] == session]
    
    # Preparar los datos de entrada y salida
    X_session = session_data[['horizontal', 'vertical']].values
    y_session = to_categorical(session_data['label'].values)
    
    # Dividir los datos en entrenamiento y prueba
    X_train_session, X_test_session, y_train_session, y_test_session = train_test_split(
        X_session, y_session, test_size=0.2, random_state=42
    )
    
    # Redimensionar los datos para la LSTM
    X_train_session = X_train_session.reshape(X_train_session.shape[0], 1, X_train_session.shape[1])
    X_test_session = X_test_session.reshape(X_test_session.shape[0], 1, X_test_session.shape[1])
    
    # Construir el modelo LSTM
    model = Sequential()
    model.add(LSTM(units=50, activation='relu', input_shape=(1, 2)))
    model.add(Dense(units=y_session.shape[1], activation='softmax'))  # Ajustar al número de categorías
    
    # Compilar el modelo
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Entrenar el modelo
    model.fit(X_train_session, y_train_session, epochs=5, batch_size=10, validation_data=(X_test_session, y_test_session))
    
    # Evaluar el modelo en los datos de prueba
    loss, accuracy = model.evaluate(X_test_session, y_test_session)
    print(f'Sesión {session}: Loss: {loss}, Accuracy: {accuracy}')
    
    # Almacenar el modelo entrenado en la lista
    trained_models_by_session.append(model)

Epoch 1/5
500/500 [==============================] - 9s 10ms/step - loss: 0.2763 - accuracy: 0.9970 - val_loss: 0.0139 - val_accuracy: 1.0000
Epoch 2/5
500/500 [==============================] - 5s 9ms/step - loss: 0.0058 - accuracy: 1.0000 - val_loss: 0.0026 - val_accuracy: 1.0000
Epoch 3/5
500/500 [==============================] - 5s 10ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 9.7881e-04 - val_accuracy: 1.0000
Epoch 4/5
500/500 [==============================] - 5s 9ms/step - loss: 6.6276e-04 - accuracy: 1.0000 - val_loss: 4.8723e-04 - val_accuracy: 1.0000
Epoch 5/5
40/40 [==============================] - 0s 5ms/step - loss: 2.7819e-04 - accuracy: 1.0000
Sesión 2: Loss: 0.0002781942894216627, Accuracy: 1.0
Epoch 1/5
285/285 [==============================] - 5s 8ms/step - loss: 0.5659 - accuracy: 1.0000 - val_loss: 0.0772 - val_accuracy: 1.0000
Epoch 2/5
285/285 [==============================] - 2s 6ms/step - loss: 0.0288 - accuracy: 1.0000 - val_loss: 0.0112 - val_acc

In [33]:
print(trained_models_by_session)

[<keras.src.engine.sequential.Sequential object at 0x0000018BCD6B72B0>, <keras.src.engine.sequential.Sequential object at 0x0000018BF6383340>, <keras.src.engine.sequential.Sequential object at 0x0000018B8ACFA910>, <keras.src.engine.sequential.Sequential object at 0x0000018BF9AB6460>, <keras.src.engine.sequential.Sequential object at 0x0000018BF9B032E0>, <keras.src.engine.sequential.Sequential object at 0x0000018B89A78F70>, <keras.src.engine.sequential.Sequential object at 0x0000018BFC2A3910>, <keras.src.engine.sequential.Sequential object at 0x0000018BF5C7C850>, <keras.src.engine.sequential.Sequential object at 0x0000018BFE851AF0>, <keras.src.engine.sequential.Sequential object at 0x0000018BB62EEDC0>, <keras.src.engine.sequential.Sequential object at 0x0000018BB62EEA90>, <keras.src.engine.sequential.Sequential object at 0x0000018C0A3FACD0>, <keras.src.engine.sequential.Sequential object at 0x0000018BFBC17250>, <keras.src.engine.sequential.Sequential object at 0x0000018C0EC175B0>, <kera

In [34]:
# Guardar el modelo entrenado
model.save('modelo_lstm_entrenado2.h5')


c:\Users\Asus\Documents\aUniversidad\SEMESTRE 23-2\Bio_I\Proyecto\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
